In [1]:
import numpy as np
from qiskit_optimization import QuadraticProgram
from qiskit.quantum_info import SparsePauliOp
from qiskit_optimization.converters import (InequalityToEquality, IntegerToBinary, 
                                            LinearEqualityToPenalty, LinearInequalityToPenalty)
from docplex.mp.model import Model

# SciPy minimizer routine
from scipy.optimize import minimize

In [2]:
class QuadraticProgram2Ising:
    """A mapping of `QuadraticProgram` to a Ising Hamiltonian
    in `SparsePauliOp` format
    """
    def __init__(self):
        self.input_type = QuadraticProgram
        self.output_type = SparsePauliOp

    def run(self, program):
        if not isinstance(program, self.input_type):
            raise TypeError(f"Program is invalid input type, must be {self.input_type}")
        ising, constant = program.to_ising(opflow=False)
        return ising + SparsePauliOp.from_list([("I"*ising.num_qubits, constant)])

In [3]:
import copy

class CompositeWorkflow:
    def __init__(self, passes):
        self.passes = passes
    
    def run(self, input):
        temp = copy.deepcopy(input)
        for individual_pass in self.passes:
            temp = individual_pass.run(temp)
        return temp

In [4]:
mdl = Model("docplex model")
x = mdl.binary_var("x")
y = mdl.binary_var("y")
z = mdl.binary_var("z")
mdl.minimize(x - 2*y + 3*z + x*y + x*z + 2*y*z)
#print(mdl.export_as_lp_string())

In [5]:
qp = QuadraticProgram.from_docplex_mp(mdl)

In [6]:
cw = CompositeWorkflow([InequalityToEquality(), # Transformation
                        IntegerToBinary(), # Transformation
                        LinearEqualityToPenalty(), # Transformation
                        LinearInequalityToPenalty(), # Transformation
                        QuadraticProgram2Ising() # Conversion
                        ])

In [7]:
ising = cw.run(qp)

In [8]:
ising

SparsePauliOp(['IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'III'],
              coeffs=[-1.  +0.j,  0.25+0.j, -2.25+0.j,  0.25+0.j,  0.25+0.j,  0.5 +0.j,
  2.  +0.j])

In [9]:
from qiskit.circuit.library import QAOAAnsatz

In [10]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session

In [11]:
service = QiskitRuntimeService()

In [12]:
backend = service.get_backend('ibmq_qasm_simulator')

In [13]:
ansatz = QAOAAnsatz(ising, reps=2)

In [14]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost


In [15]:
session = Session(backend=backend)

estimator = Estimator(session=session, options={"shots": int(1e4)})
sampler = Sampler(session=session, options={"shots": int(1e4)})


In [16]:
x0 = 2 * np.pi * np.random.rand(ansatz.num_parameters)

In [ ]:
res = minimize(cost_func, x0, args=(ansatz, ising, estimator), method="COBYLA")

In [ ]:
res

In [ ]:
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
# Add measurements to our circuit
qc.measure_all()


In [ ]:
# Sample ansatz at optimal parameters
samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
# Close the session since we are now done with it
session.close()

In [ ]:
samp_dist

In [ ]:
probs = samp_dist.binary_probabilities()
probs

In [356]:
def evaluate_quadratic_program(bitstring, program):
    constant = program.objective.constant
    linear_elements = program.objective.linear.to_dict()
    quadratic_elements = program.objective._quadratic.to_dict()

    x = np.fromiter(list(bitstring[::-1]), dtype=int)
    
    sum = constant
    for element, val in linear_elements.items():
        sum += x[element] * val
    for element, val in quadratic_elements.items():
        sum += x[element[0]] * val * x[element[1]]
    return sum

In [357]:
results = []
for bitstring in probs:
    results.append(evaluate_quadratic_program(bitstring, qubo))
    
results

[5.0, 1.0, -2.0, 3.0, 3.0, 0.0, 0.0, 6.0]

In [358]:
qubo.variables

[<Variable: x (binary)>, <Variable: y (binary)>, <Variable: z (binary)>]